In [ ]:
%pip install pytesseract
%pip install opencv-python
%pip install scrcpy-client[ui]
%pip install Pillow
%pip install cv2
%pip install numpy
%pip install pandas
%pip install tqdm
pip install ipywidgets



In [68]:
import os
import time
from PIL import Image
from subprocess import check_output
import re
import sys
import cv2
import pytesseract
from datetime import datetime
import numpy as np
from scrcpy import Client
from scrcpy import const
from IPython.display import display,clear_output
import timeit
import pandas as pd
import random
from tqdm.notebook import trange, tqdm

class Bot:
    work = 1
    screenshot = 0
    screenshotName = ''
    t1 = 0
    fight = 1
    device = 0
    coopMode = True
    countMana = 0
    runCount = 0


    def main(self):
        self.getScreen()

    def start(self,device):
        self.device = device #'127.0.0.1:'+
        self.screenshotName = self.device + '-screenshot.png'
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'
        self.work = 1
        self.client = Client(device='emulator-5554')
        self.client.start(threaded=True)
        time.sleep(0.5)

    def stop(self):
        self.client.stop()

    # Function to send ADB shell command
    def shell(self, cmd):
        os.system(f'"C:/Programs/Scrcpy/adb" -s {self.device} shell {cmd}')

    # Send ADB to click screen
    def click(self, x, y, timer=True):
        self.client.control.touch(x, y, const.ACTION_DOWN)
        #self.shell(f'input tap {x} {y}')
        if (timer):
            time.sleep(0.2)
        self.client.control.touch(x, y, const.ACTION_UP)

    def swipe(self,direction):
        if not direction in ['left','right','up','down']:
            return
        boxes,box_size = get_grid()
        if direction=='left':
            self.client.control.swipe(*boxes[1,0],*boxes[2,0])
        elif direction=='right':
            self.client.control.swipe(*boxes[2,0],*boxes[1,0])
        elif direction=='up':
            self.client.control.swipe(*boxes[0,2],*boxes[0,1])
        elif direction=='down':
            self.client.control.swipe(*boxes[0,1],*boxes[0,2])

    # Send ADB command to click E
    def keyE(self):
        self.client.control.keycode(33)
    


    # Take screenshot of device screen and load pixel values 
    def getScreen(self):
        self.shell(f'/system/bin/screencap -p /sdcard/{self.screenshotName}')
        # Using the adb command to upload the screenshot of the mobile phone to the current directory
        os.system(f'"C:/Programs/Scrcpy/adb" -s {self.device} pull /sdcard/{self.screenshotName}')
        try:
            self.screenshot = Image.open(f"{self.screenshotName}")
        except ValueError:
            print(ValueError)
            self.getScreen()
            
    # Check pixel color at x,y position
    def getPixelColor(self, x1, y1):
        self.getScreen()
        im = Image.open(f"{self.screenshotName}")
        # im1 = ImageOps.crop(im, (0, 0, 1000, 300))
        # im1.show()
        pixelRGB = im.getpixel((x1, y1))[:3]
        return pixelRGB
    def crop_img(self, x, y, dx, dy, name='timeTrain.png'):
        # Load screen
        img_rgb = cv2.imread(self.screenshotName)
        img_rgb = img_rgb[y:y + dy, x:x + dx]
        # Convert to grayscale (done internally by tessarct )
        #(a, img_gray) = cv2.threshold(img_rgb, 127, 255, cv2.THRESH_BINARY)
        cv2.imwrite(name, img_rgb)

    def getText(self, x, y, dx, dy,new=True):
        if new: self.getScreen()
        # crop image
        self.crop_img(x, y, dx, dy)
        # Do OCR with Google Tesseract
        strTime = pytesseract.image_to_string('timeTrain.png',config='--psm 13').replace('\n', '')#.replace(' ', '')
        # improved digits 
        # pytesseract.image_to_string('timeTrain.png',config='--psm 13 --oem 3 -c tessedit_char_whitelist=0123456789')
        return (strTime)

    def scan_grid(self):
        boxes,box_size = get_grid()
        # manually get screenshot
        self.getScreen()
        box_list = boxes.reshape(16,2)
        names=[]
        for i in range(len(box_list)):
            file_name = f'OCR_inputs/timeTrain_{str(i)}.png'
            bot.crop_img(*box_list[i],*box_size,name=file_name)
            names.append(file_name)
        df = pd.DataFrame(names, columns=["filenames"])
        df.to_csv('ocr_input.txt', index=False,header=False)
        grid_content = pytesseract.image_to_string('ocr_input.txt',config='--psm 13 --oem 3 -c tessedit_char_whitelist=0123456789')
        # Split up
        grid_content = grid_content.replace('\n', '').split('\x0c')
        grid_content=np.array(grid_content).reshape(4,4)
        self.grid_content=np.transpose(grid_content)
        return self.grid_content


    def getXYByImage(self, target):
        valid_targets = ['youtube','facebook','instagram']
        if not target in valid_targets: 
            return "INVALID TARGET" 
        self.getScreen()
        imgSrc=f'icons/{target}.png'
        img_rgb = cv2.imread(f"{self.screenshotName}")
        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
        template = cv2.imread(imgSrc, 0)
        res = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        threshold = 0.8
        loc = np.where(res >= threshold)
        if len(loc[0]) > 0:
            y = loc[0][0]
            x = loc[1][0]
            return [x, y]
    def getTime(self):
        return self.getText(x=1530,y=5,dx=70,dy=25)
    def get_searchBox(self):
        return self.getText(x=275,y=120,dx=500,dy=40)

def get_grid():
    #Grid dimensions
    top_box=(60,500)
    box_size=(180,180)
    gap=20
    length=4
    # x_cords
    x_cord=list(range(top_box[0],top_box[0]+box_size[0]*length,box_size[0]+gap))
    y_cord=list(range(top_box[1],top_box[1]+box_size[1]*length,box_size[1]+gap))
    boxes=[]
    # Create list of all boxes
    for x_point in x_cord:
        for y_point in y_cord:
            boxes.append((x_point,y_point))
    # Convert to np array (4x4) with x,y coords
    boxes = np.array(boxes).reshape(length,length,2)
    return boxes, box_size
boxes,box_size = get_grid()

def find_device():
    devList = check_output('"C:/Programs/Scrcpy/adb" devices')
    devListArr = str(devList).split('\\n')
    # Check for online status
    for client in devListArr[1:]:
        if 'device' in client:
            # split IP from status and store device
            deivce = client.split('\\t')[0]
            print("Found ADB device! {}".format(deivce))
            return deivce
    return None
    
device = find_device()

bot = Bot()
bot.start(device)


Found ADB device! emulator-5554


Invalid NAL unit 0, skipping.
 (repeated 12 more times)
Invalid NAL unit 0, skipping.


In [2]:
bot.stop()

In [120]:
board_log=[]
score=0
for i in range(1000):
    # take random action
    action = random.choice( ['left','right','up','down'])
    bot.swipe(action)
    # read board
    df = pd.DataFrame(bot.scan_grid(), columns=['0','1','2','3'])
    board_log.append(df)
    display(df,clear=True)
    print("Move: {}\nLast Action: {}\nLast Score: {}".format(i,action,score))
    if i%5==4:
        score = bot.getText(520,260,120,70) # score window 2048
        # check if game over
        if df.equals(board_log[-4]):
            bot.click(800,400) #restart button
            time.sleep(1)
            bot.click(290,1040) #confirm
        


,0,1,2,3
0,,,,2
1,2,2,,4
2,4,16,8,2
3,32,8,16,8


Move: 54
Last Action: down
Last Score: 192,


KeyboardInterrupt: 